<span style="color:DarkBlue;font-size:18px">Semesterabschließende schriftliche Ausarbeitung im Modul Machine Learning (SS2023)</span>

<span style="color:DarkBlue;font-size:32px">Preisvorhersage von Gebrauchtwagen </span>

<span style="color:DarkBlue;font-size:18px"> Bearbeitet von M.Sc. Onur Yilmaz</span>

### **Inhaltsverzeichnis**

#### **Einleitung**

#### **1. Datenaufbereitung**
#### **2. Explorative Datenanalyse**
#### **3. Feature Engineering**
#### **4. Modell Auswahl und Training**
#### **5. Modell Bewertung**
#### **6. Streamlit** 
#### **Literaturverzeichnis**




### **Einleitung**

In der vorliegenden semesterabschließenden Arbeit wird ein aufbereiteter Datensatz von 100.000 gebrauchten Autoanzeigen aus dem Vereinigten Königreich analysiert. Der Datensatz wurde von der Plattform Kaggle bezogen - https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes.

Das Hauptziel dieser Arbeit ist die Wahl und Implementierung eines geeigneten maschinellen Lernmodells zur Preisvorhersage von Gebrauchtwagen.

Die Arbeit gliedert sich in verschiedene Abschnitte, beginnend mit der Datenaufbereitung, gefolgt von der explorativen Datenanalyse und dem Feature Engineering. Anschließend wird das passende Modell ausgewählt und trainiert, bevor es bewertet wird. Zusätzlich wird die Implementierung in Streamlit durchgeführt, um eine interaktive Webanwendung zur Darstellung der Ergebnisse zu erstellen.

Der Datensatz ist in einzelne Dateien nach Autoherstellern unterteilt und enthält Informationen wie Preis, Getriebe, Kilometerstand, Kraftstoffart, Kfz-Steuer, Verbrauch in Meilen pro Gallone (mpg) und Motorgröße.

*Das gesamte Projekt kann aus dem folgenden Repository geklont werden: https://www.github.com/ONURYI*

### **1. Datenaufbereitung**

Die Datenaufbereitung stellt im Prozess des maschinellen Lernens oft einen entscheidenden, aber zugleich zeitaufwendigen Schritt dar. Sie umfasst diverse Techniken zur Säuberung, Transformation und Organisation der Rohdaten, um sie in ein für die Analyse passendes Format zu überführen. Diese Etappe legt das solide Fundament für die Erstellung eines Modells, da sie die Qualität und Relevanz der Daten gewährleistet. Dies trägt maßgeblich zur Präzision und Leistungsfähigkeit des endgültigen Modells bei [2].

Im Kontext unseres Projektes verwenden wir folgende CSV-Dateien (CSV = Comma-Separated Values):

- audi.csv
- bmw.csv
- ford.csv
- hyundai.csv
- mercedes.csv
- skoda.csv
- toyota.csv
- vauxhall.csv
- vw.csv

Diese Dateien sind im Ordner **/data** gespeichert und werden nach dem Aufbereitungs- und Bereinigungsprozess im Ordner **/data (clean)** als einzige zusammengefasste CSV-Datei abgelegt.

Für diesen Teil des Projekts wird ausschließlich auf die weit verbreitete Pandas-Bibliothek zurückgegriffen, welche sich als leistungsfähiges Werkzeug zur Datenmanipulation und -analyse etabliert hat.

In [3]:
import pandas as pd

In [4]:
# Liste der Dateipfade mit dem relativen Pfad '../'
file_paths = [
    '../data/audi.csv',
    '../data/bmw.csv',
    '../data/ford.csv',
    '../data/hyundi.csv',
    '../data/mercedes.csv',
    '../data/skoda.csv',
    '../data/toyota.csv',
    '../data/vauxhall.csv',
    '../data/vw.csv',
]

# Liste, um alle DataFrames zu speichern
all_dataframes = []

# Durchlaufe alle Dateipfade und lade die CSV-Dateien
for file_path in file_paths:
    df = pd.read_csv(file_path)
    brand = file_path.split('/')[-1].split('.')[0]  # Extrahiere die Marke aus dem Dateinamen
    df['brand'] = brand  # Füge eine Spalte mit der Marke hinzu
    # Ändere die Reihenfolge der Spalten, um "brand" an den Anfang zu setzen
    df = df[['brand'] + [col for col in df.columns if col != 'brand']]
    all_dataframes.append(df)

# Verbinde alle DataFrames
df = pd.concat(all_dataframes, ignore_index=True)

In [5]:
df.head()

,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,audi,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN
1,audi,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN
2,audi,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN
3,audi,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN
4,audi,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN


### **Literaturverzeichnis**

[1] https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes (01.08.2023)

[2] Müller, A. C., & Guido, S. (2021). Introduction to Machine Learning with Python: A Guide for Data Scientists (2nd ed.). O'Reilly Media